# Imports e config

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from dataclasses import dataclass, fields, asdict

import time

In [35]:
driver = webdriver.Chrome()
driver.implicitly_wait(4)
wait = WebDriverWait(driver, 10)

# Config site

In [36]:
URL_BASE = 'https://www.transparencia.sc.gov.br/remuneracao-servidores'
driver.get(URL_BASE)

In [ ]:
class Scraping():
    ''' Classe para alternar meses e páginas '''
    
    def __init__(self,ano):
        ''' Scraping do ano escolhido '''
        self.quantidade_paginas = 0
        self.ano = ano
    
    def quantidade_mes():
        '''
        Retorna a quantidade de meses
        '''
        mes = driver.find_element(By.CSS_SELECTOR, '#select-mes').get_property('children')
        return len(mes)
    
    def selecionar_mes(self, index):
        '''
        Seleciona o mês de acordo com o index e retorna qual mês foi selecionado
        '''
        mes = Select(driver.find_element(By.CSS_SELECTOR, '#select-mes'))
        mes.select_by_index(index)
        mes_atual = driver.find_element(By.CSS_SELECTOR, '#select-mes').get_property('children')[index].text
        return mes_atual
    
    def selecionar_ano(self):
        '''
        Seleciona o ano de acordo com a string específicada ex: "2025"
        '''
        ano = Select(driver.find_element(By.CSS_SELECTOR, '#select-ano'))
        print(f'Ano selecionado: {self.ano}')
        ano.select_by_visible_text(f'{self.ano}')
    
    def botao_buscar(self):
        ''' Clica em buscar servidores '''
        
        lupa = driver.find_element(By.CSS_SELECTOR, '.botao-buscar-servidor')
        ActionChains(driver).scroll_to_element(lupa).perform()
        lupa.click()
    
    
    
    


In [19]:
scrapy = Scraping(2025)
scrapy.selecionar_mes(1)

'Fevereiro'

In [33]:
teste = Scraping(2025)

In [34]:
teste.botao_buscar()

# API SITE

O site contém API para os dados podemos coletar dados com eles 

In [41]:
import requests
import pandas as pd
import time
import random
import os
import json
from datetime import datetime
from fake_useragent import UserAgent

# Configurações de Caminho
BASE_DIR = os.path.join("data", "base")
LOG_FILE = os.path.join(BASE_DIR, "scraping_log_2025.txt")
BASE_URL = "https://api-portal-transparencia.apps.sm.okd4.ciasc.sc.gov.br/api/remuneracao-servidores/analise-detalhada"
PAGES_PER_CHECKPOINT = 50

# Garante que a pasta existe
os.makedirs(BASE_DIR, exist_ok=True)

def get_status_progresso():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            lines = f.readlines()
            if lines:
                last_line = lines[-1].strip()
                if "FIM:" in last_line:
                    partes = last_line.split("|")
                    mes = partes[0].split("Mes")[-1].strip()
                    pag = partes[1].split("Pagina")[-1].strip()
                    return mes, int(pag)
    return "01", 0

def save_to_csv(data_list, mes):
    filename = os.path.join(BASE_DIR, f"transparencia_sc_2025_{mes}.csv")
    df = pd.DataFrame(data_list)
    is_new = not os.path.exists(filename)
    df.to_csv(filename, mode='a', index=False, header=is_new, encoding='utf-8-sig')

def run_scraper():
    ua = UserAgent()
    session = requests.Session()
    
    ultimo_mes_log, ultima_pag_log = get_status_progresso()
    meses_2025 = [str(m).zfill(2) for m in range(1, 13)]
    meses_restantes = meses_2025[meses_2025.index(ultimo_mes_log):]

    print(f"{'='*70}")
    print(f"🚀 SCRAPER EM EXECUÇÃO - DESTINO: /data/base/")
    print(f"📍 Status: Iniciando em Mês {ultimo_mes_log} | Página {ultima_pag_log}")
    print(f"{'='*70}")

    try:
        for mes in meses_restantes:
            current_page = (ultima_pag_log + 1) if mes == ultimo_mes_log else 1
            last_page_limit = None
            temp_storage = []
            
            while True:
                headers = {
                    "User-Agent": ua.random,
                    "Referer": "https://www.transparencia.sc.gov.br/",
                    "Accept": "application/json"
                }
                
                params = {
                    "ano": 2025,
                    "mes": mes,
                    "page": current_page,
                    "filtro": json.dumps({"TIPOCONSULTA": "SERVIDORES", "nome": ""}),
                    "sort": json.dumps({"field": "ServidorNomeSort", "dir": "asc"})
                }

                try:
                    t_start = time.time()
                    response = session.get(BASE_URL, params=params, headers=headers, timeout=30)
                    response.raise_for_status()
                    json_data = response.json()
                    
                    if last_page_limit is None:
                        last_page_limit = json_data.get('last_page', 1)
                        print(f"\n[INFO] Mês {mes}/2025: {last_page_limit} páginas encontradas.")

                    batch = json_data.get('data', [])
                    
                    # INJEÇÃO DE DADOS (Mês e Ano)
                    for item in batch:
                        item['ref_mes'] = mes
                        item['ref_ano'] = 2025

                    temp_storage.extend(batch)
                    
                    # Dashboard de progresso no terminal
                    print(f"📡 [Mês {mes}] Pág: {current_page}/{last_page_limit} | "
                          f"Itens: {len(batch)} | Time: {time.time() - t_start:.2f}s", end='\r')

                    # Salva a cada X páginas ou na última do mês
                    if current_page % PAGES_PER_CHECKPOINT == 0 or current_page == last_page_limit:
                        save_to_csv(temp_storage, mes)
                        
                        with open(LOG_FILE, "a") as f:
                            f.write(f"FIM: Mes {mes} | Pagina {current_page}\n")
                        
                        print(f"\n💾 CHECKPOINT: Mês {mes}, Pág {current_page} salva em /data/base/")
                        temp_storage = []

                    if current_page >= last_page_limit:
                        print(f"\n✅ Mês {mes} finalizado!")
                        break
                    
                    current_page += 1
                    time.sleep(random.uniform(0, 0.5))

                except Exception as e:
                    print(f"\n🚨 Erro na Pág {current_page}: {str(e)[:100]}")
                    time.sleep(30)
                    continue
            
            ultima_pag_log = 0 

    except KeyboardInterrupt:
        print(f"\n\n🛑 Interrompido pelo usuário.")
    finally:
        if temp_storage:
            save_to_csv(temp_storage, mes)
        print(f"\n🏁 Fim da execução. Logs em: {LOG_FILE}")

if __name__ == "__main__":
    run_scraper()

🚀 SCRAPER EM EXECUÇÃO - DESTINO: /data/base/
📍 Status: Iniciando em Mês 01 | Página 450

[INFO] Mês 01/2025: 7158 páginas encontradas.
📡 [Mês 01] Pág: 500/7158 | Itens: 20 | Time: 0.30s
💾 CHECKPOINT: Mês 01, Pág 500 salva em /data/base/
📡 [Mês 01] Pág: 550/7158 | Itens: 20 | Time: 0.43s
💾 CHECKPOINT: Mês 01, Pág 550 salva em /data/base/
📡 [Mês 01] Pág: 600/7158 | Itens: 20 | Time: 0.32s
💾 CHECKPOINT: Mês 01, Pág 600 salva em /data/base/
📡 [Mês 01] Pág: 650/7158 | Itens: 20 | Time: 0.29s
💾 CHECKPOINT: Mês 01, Pág 650 salva em /data/base/
📡 [Mês 01] Pág: 700/7158 | Itens: 20 | Time: 0.28s
💾 CHECKPOINT: Mês 01, Pág 700 salva em /data/base/
📡 [Mês 01] Pág: 750/7158 | Itens: 20 | Time: 0.39s
💾 CHECKPOINT: Mês 01, Pág 750 salva em /data/base/
📡 [Mês 01] Pág: 791/7158 | Itens: 20 | Time: 0.24s

🛑 Interrompido pelo usuário.

🏁 Fim da execução. Logs em: data\base\scraping_log_2025.txt


In [ ]:
import requests
import pandas as pd
import time
import random
import os
import json
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from fake_useragent import UserAgent

# Configurações de Caminho
BASE_DIR = os.path.join("data", "details")
os.makedirs(BASE_DIR, exist_ok=True)
BASE_URL = "https://api-portal-transparencia.apps.sm.okd4.ciasc.sc.gov.br/api/remuneracao-servidores/analise-detalhada"

def get_last_page_from_log(mes):
    """Lê o progresso específico de um mês."""
    log_path = os.path.join(BASE_DIR, f"log_mes_{mes}.txt")
    if os.path.exists(log_path):
        with open(log_path, "r") as f:
            line = f.read().strip()
            if line:
                return int(line)
    return 0

def salvar_checkpoint_mes(mes, pagina):
    """Registra a última página processada para evitar recomeço."""
    log_path = os.path.join(BASE_DIR, f"log_mes_{mes}.txt")
    with open(log_path, "w") as f:
        f.write(str(pagina))

def coletar_mes(mes):
    """Função que será executada em paralelo para cada mês."""
    ua = UserAgent()
    session = requests.Session()
    
    ultima_pag = get_last_page_from_log(mes)
    current_page = ultima_pag + 1
    last_page_limit = None
    temp_storage = []
    
    # Print inicial detalhado
    status = "RETOMANDO" if ultima_pag > 0 else "INICIANDO"
    print(f"📡 [{status}] Mês {mes}/2025 | Começando da Pág: {current_page}")

    while True:
        headers = {
            "User-Agent": ua.random,
            "Referer": "https://www.transparencia.sc.gov.br/",
            "Accept": "application/json"
        }
        
        params = {
            "ano": 2025,
            "mes": mes,
            "page": current_page,
            "filtro": json.dumps({"TIPOCONSULTA": "SERVIDORES", "nome": ""}),
            "sort": json.dumps({"field": "ServidorNomeSort", "dir": "asc"})
        }

        try:
            t_start = time.time()
            response = session.get(BASE_URL, params=params, headers=headers, timeout=30)
            response.raise_for_status()
            json_data = response.json()
            t_end = time.time()
            
            if last_page_limit is None:
                last_page_limit = json_data.get('last_page', 1)
                # Verifica se o mês já estava completo
                if current_page > last_page_limit:
                    print(f"✅ [Mês {mes}] Já estava 100% concluído anteriormente.")
                    break

            batch = json_data.get('data', [])
            
            # Injeção de metadados para sua análise como Data Analyst
            for item in batch:
                item['ref_mes'] = mes
                item['ref_ano'] = 2025
            
            temp_storage.extend(batch)

            # Print de andamento por página
            prog = (current_page / last_page_limit) * 100
            print(f"▶️ [Mês {mes}] Pág: {current_page}/{last_page_limit} ({prog:.1f}%) | Latência: {t_end - t_start:.2f}s")

            # Checkpoint a cada 50 páginas
            if current_page % 50 == 0 or current_page == last_page_limit:
                filename = os.path.join(BASE_DIR, f"transparencia_sc_2025_{mes}.csv")
                df = pd.DataFrame(temp_storage)
                df.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename), encoding='utf-8-sig')
                
                salvar_checkpoint_mes(mes, current_page)
                
                print(f"💾 [Mês {mes}] CHECKPOINT: {len(temp_storage)} itens salvos. (Hora: {datetime.now().strftime('%H:%M:%S')})")
                temp_storage = [] 

            if current_page >= last_page_limit:
                print(f"🏆 [Mês {mes}] FINALIZADO COM SUCESSO!")
                break
            
            current_page += 1
            # Delay agressivo conforme seu teste bem-sucedido
            time.sleep(random.uniform(1, 2.5)) 

        except Exception as e:
            print(f"🚨 [Mês {mes}] ERRO na Pág {current_page}: {str(e)[:100]}")
            time.sleep(20) # Pausa para o servidor respirar
            continue

def run_multithreaded():
    meses = [str(m).zfill(2) for m in range(1, 13)]
    
    print(f"{'='*70}")
    print(f"🔥 INICIANDO EXTRAÇÃO PARALELA (4 THREADS) - AGRESSIVE MODE")
    print(f"📁 Destino: {BASE_DIR}")
    print(f"{'='*70}")
    
    # max_workers=4 é o ideal para o seu processador Intel equilibrar velocidade e estabilidade
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(coletar_mes, meses)

if __name__ == "__main__":
    start_total = time.time()
    try:
        run_multithreaded()
    except KeyboardInterrupt:
        print(f"\n🛑 Interrupção manual detectada pelo usuário.")
    finally:
        print(f"\n{'='*70}")
        print(f"✨ TUDO PRONTO! Tempo Total: {time.time() - start_total:.2f}s")
        print(f"{'='*70}")

🔥 INICIANDO EXTRAÇÃO PARALELA (4 THREADS) - AGRESSIVE MODE
📁 Destino: data\details
📡 [RETOMANDO] Mês 04/2025 | Começando da Pág: 151
📡 [RETOMANDO] Mês 03/2025 | Começando da Pág: 151
📡 [RETOMANDO] Mês 02/2025 | Começando da Pág: 151
📡 [RETOMANDO] Mês 01/2025 | Começando da Pág: 151
▶️ [Mês 02] Pág: 151/7994 (1.9%) | Latência: 0.48s
▶️ [Mês 01] Pág: 151/7158 (2.1%) | Latência: 0.48s
▶️ [Mês 04] Pág: 151/9153 (1.6%) | Latência: 0.49s
▶️ [Mês 03] Pág: 151/8886 (1.7%) | Latência: 0.49s
▶️ [Mês 02] Pág: 152/7994 (1.9%) | Latência: 0.20s
▶️ [Mês 01] Pág: 152/7158 (2.1%) | Latência: 0.17s
▶️ [Mês 03] Pág: 152/8886 (1.7%) | Latência: 0.29s
▶️ [Mês 04] Pág: 152/9153 (1.7%) | Latência: 0.19s
▶️ [Mês 01] Pág: 153/7158 (2.1%) | Latência: 0.15s
▶️ [Mês 02] Pág: 153/7994 (1.9%) | Latência: 0.17s
▶️ [Mês 04] Pág: 153/9153 (1.7%) | Latência: 0.18s
▶️ [Mês 03] Pág: 153/8886 (1.7%) | Latência: 0.19s
▶️ [Mês 03] Pág: 154/8886 (1.7%) | Latência: 0.32s
▶️ [Mês 04] Pág: 154/9153 (1.7%) | Latência: 0.21s
▶️ 